In [1]:
import sys
print(sys.executable)


c:\Users\Gur Levy\heinecan-4\.venv\Scripts\python.exe


In [2]:
!pip install requests


In [3]:
import requests
import csv
from datetime import datetime, timedelta


In [4]:
# api endpoint and params
url = 'https://api.openweathermap.org/data/3.0/onecall'
params = {
    'lat': 25.686613,
    'lon': -100.316116,
    'exclude': 'current,minutely,daily,alerts',
    'appid': 'd420319993e5906012bee411669261e0',
    'units': 'metrics'
}
#api request
response = requests.get(url, params = params)
data = response.json()
#timezone offset
offset = data.get('timezone_offset', 0)
#csv file
csv_rows = []
header = ['datetime', 'temp',  'feels_like', 'pressure', 'humidity', 'dew_point', 'uvi', 'clouds','wind_speed', 'wind_deg', 'weather_description']
#changing to datetime
for entry in data['hourly']:
    dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)
    weather_desc = entry['weather'][0]['description'] if entry.get('weather')else''
    row = [
        dt.strftime("%Y-%m-%d %H:%M:%S"),
        entry.get("temp"),
        entry.get("feels_like"),
        entry.get("pressure"),
        entry.get("humidity"),
        entry.get("dew_point"),
        entry.get("uvi"),
        entry.get("clouds"),
        entry.get("wind_speed"),
        entry.get("wind_deg"),
        weather_desc
    ]
    csv_rows.append(row)

#csv save
csv_file = 'monterrey_hourly_weather.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(csv_rows)

print(f'file saved{csv_file}')


file savedmonterrey_hourly_weather.csv


C:\Users\Gur Levy\AppData\Local\Temp\ipykernel_5068\2351409520.py:20: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)


In [5]:
import requests
import csv
from datetime import datetime, timedelta

# monterrey location and api 
lat = 25.686613
lon = -100.316116
api_key = 'd420319993e5906012bee411669261e0'

# hourly weather from One Call API
weather_url = 'https://api.openweathermap.org/data/3.0/onecall'
weather_params = {
    'lat': lat,
    'lon': lon,
    'exclude': 'current,minutely,daily,alerts',
    'appid': api_key,
    'units': 'metric' 
}

weather_response = requests.get(weather_url, params=weather_params)
weather_data = weather_response.json()
offset = weather_data.get('timezone_offset', 0)

# build dict by datetime string (hour) for easy merging
weather_by_hour = {}
for entry in weather_data['hourly']:
    dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)
    dt_str = dt.strftime("%Y-%m-%d %H")
    weather_by_hour[dt_str] = {
        'datetime': dt.strftime("%Y-%m-%d %H:%M:%S"),
        'temp': entry.get("temp"),
        'humidity': entry.get("humidity"),
        'wind_speed': entry.get("wind_speed")
    }

# hourly solar radiation from Solar API 
solar_url = 'https://api.openweathermap.org/energy/1.0/solar/data'
solar_by_hour = {}

for i in range(7):
    date_str = (datetime.now() + timedelta(days=i)).strftime('%Y-%m-%d')
    params = {
        'lat': lat,
        'lon': lon,
        'date': date_str,
        'appid': api_key
    }

    response = requests.get(solar_url, params=params)
    if response.status_code == 200:
        data = response.json()
        hourly = data['irradiance']['hourly']
        for h in hourly:
            dt_str = f"{date_str} {h['hour']:02}"
            solar_by_hour[dt_str] = h['clear_sky']['ghi']  # W/m²
    else:
        print(f"Solar API failed for {date_str}: {response.status_code}")

# merge and create final csv
final_rows = []
header = ['datetime', 'temp', 'humidity', 'wind_speed', 'solar_ghi']

for dt_hour, weather in weather_by_hour.items():
    solar = solar_by_hour.get(dt_hour)
    if solar is not None:
        row = [
            weather['datetime'],
            weather['temp'],
            weather['humidity'],
            weather['wind_speed'],
            solar
        ]
        final_rows.append(row)

# save file
csv_file = 'monterrey_utci_inputs.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(final_rows)

print(f'UTCI input file saved: {csv_file}')


C:\Users\Gur Levy\AppData\Local\Temp\ipykernel_5068\303758452.py:27: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)


UTCI input file saved: monterrey_utci_inputs.csv


In [6]:
!pip install pandas

In [ ]:
import pandas as pd

# merged data
df = pd.read_csv('monterrey_utci_inputs.csv')

# estimate MRT (mean radiant temperature)
# MRT = temp + 0.03 * GHI 
df['mrt'] = df['temp'] + 0.03 * df['solar_ghi']

# this is NOT the official UTCI model, empirical regression
# Bröde et al. (2012) — simplified terms only
df['utci'] = df['temp'] + \
    0.607562 + \
    -0.022771 * df['temp'] + \
    0.000806 * df['humidity'] + \
    -0.002 * df['wind_speed'] + \
    0.0000012 * df['temp']**2 + \
    -0.0000028 * df['humidity']**2 + \
    0.0000035 * df['wind_speed']**2 + \
    0.0006 * (df['mrt'] - df['temp'])   # added influence of radiant temperature


df.to_csv('monterrey_utci_results.csv', index=False)

print("UTCI calculation complete. File saved: monterrey_utci_results.csv")


UTCI calculation complete. File saved: monterrey_utci_results.csv
